In [1]:


import pandas as pd
from src.data.ingestion import load_tickets

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [3]:
import pandas as pd

#pd.set_option('display.max_colwidth', None)


In [4]:
json_data = load_tickets("../data/raw/support_tickets.json")

In [5]:
#json_data[:5]

In [6]:
import pandas as pd

df_data = pd.DataFrame([ticket.model_dump() for ticket in json_data[0]])
df_data.head()


,ticket_id,created_at,updated_at,customer_id,customer_tier,organization_id,product,product_version,product_module,category,...,escalated,transferred_count,satisfaction_score,resolution_helpful,tags,environment,business_impact,affected_users,language,region
0,TK-2024-000001,2023-11-02 12:30:10+00:00,2023-11-02 15:30:46+00:00,CUST-02387,starter,ORG-234,CloudBackup Enterprise,4.5.10,encryption_layer,Feature Request,...,True,0,4,True,"[error, api, integration, timeout, bug]",production,high,222,de,APAC
1,TK-2024-000002,2023-02-10 16:31:31+00:00,2023-02-12 09:59:43+00:00,CUST-03724,free,ORG-435,DataSync Pro,4.1.11,data_validator,Account Management,...,True,3,4,True,"[database, bug, authentication, data, error]",production,medium,18,ja,MEA
2,TK-2024-000003,2024-09-30 07:43:47+00:00,2024-09-30 11:58:47+00:00,CUST-00600,enterprise,ORG-208,API Gateway,3.1.4,request_router,Feature Request,...,False,3,4,True,"[configuration, error, sync, performance]",staging,medium,591,ja,NA
3,TK-2024-000004,2024-11-27 18:17:26+00:00,2024-11-30 22:07:50+00:00,CUST-04795,starter,ORG-231,CloudBackup Enterprise,3.4.15,backup_service,Account Management,...,False,2,3,False,"[authentication, api, performance]",production,critical,34,en,LATAM
4,TK-2024-000005,2024-03-09 15:41:02+00:00,2024-03-10 10:53:38+00:00,CUST-01101,starter,ORG-241,StreamProcessor,2.8.8,monitoring,Feature Request,...,False,2,5,True,"[data, integration, security, authentication]",development,medium,325,de,MEA


In [7]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110000 entries, 0 to 109999
Data columns (total 35 columns):
 #   Column              Non-Null Count   Dtype              
---  ------              --------------   -----              
 0   ticket_id           110000 non-null  object             
 1   created_at          110000 non-null  datetime64[ns, UTC]
 2   updated_at          110000 non-null  datetime64[ns, UTC]
 3   customer_id         110000 non-null  object             
 4   customer_tier       110000 non-null  object             
 5   organization_id     110000 non-null  object             
 6   product             110000 non-null  object             
 7   product_version     110000 non-null  object             
 8   product_module      110000 non-null  object             
 9   category            110000 non-null  object             
 10  subcategory         110000 non-null  object             
 11  priority            110000 non-null  object             
 12  severity        

In [8]:
df_data['category'].unique()

array(['Feature Request', 'Account Management', 'Security', 'Data Issue',
       'Technical Issue'], dtype=object)

In [9]:
df_data[df_data['category'].isnull()]

,ticket_id,created_at,updated_at,customer_id,customer_tier,organization_id,product,product_version,product_module,category,...,escalated,transferred_count,satisfaction_score,resolution_helpful,tags,environment,business_impact,affected_users,language,region


In [10]:
df_data['subject'].nunique()

110

In [11]:
df_data['description'].nunique()

350

In [12]:
df_data['category'].value_counts()

category
Security              22085
Data Issue            22050
Feature Request       22047
Account Management    21997
Technical Issue       21821
Name: count, dtype: int64

In [13]:
df_data['subcategory'].unique()

array(['Documentation', 'Upgrade', 'New Feature', 'API', 'Compliance',
       'Corruption', 'Enhancement', 'Sync Error', 'UI/UX',
       'Import/Export', 'Bug', 'Access Control', 'Vulnerability',
       'Authentication', 'Billing', 'Integration', 'Configuration',
       'Performance', 'Encryption', 'License', 'Authorization',
       'Compatibility', 'Subscription', 'Validation', 'Data Loss'],
      dtype=object)

In [14]:
from src.classification.data_processing import prepare_classification_dataset, group_text_and_train_val_test_split, build_feature_encoder_pipeline_tfidf,clean_error_logs_preserve_codes

In [15]:
### remove the date and time info from the error_logs feature
### select the features which are usefull for classification

classification_data= prepare_classification_dataset(json_data[0])
classification_data = pd.DataFrame(classification_data)
classification_data.head(2)

,text,product,product_module,priority,channel,customer_tier,region,category,subcategory,subject,description,error_logs
0,Request: Add bulk operation support to CloudBa...,CloudBackup Enterprise,encryption_layer,critical,portal,starter,APAC,Feature Request,Documentation,Request: Add bulk operation support to CloudBa...,We would like to request a feature for CloudBa...,
1,License upgrade needed for DataSync Pro\nWe ne...,DataSync Pro,data_validator,medium,chat,free,MEA,Account Management,Upgrade,License upgrade needed for DataSync Pro,We need to upgrade our license for DataSync Pr...,WARN Rate limit approaching threshold ERROR ER...


In [16]:
classification_data.shape

(110000, 12)

In [17]:
### data set is partially synthetic so I am createing the groups based on description of the issue so the we don't have the same issue in train and test sets. 

train_df, val_df, test_df = group_text_and_train_val_test_split(classification_data)

print(len(train_df), len(val_df), len(test_df))

63498 11022 11505


In [18]:
train_df.head(2)

,text,product,product_module,priority,channel,customer_tier,region,subcategory,text_key,group_id,category
1,License upgrade needed for DataSync Pro We nee...,DataSync Pro,data_validator,medium,chat,free,MEA,Upgrade,We need to upgrade our license for DataSync Pr...,9757097747101013450,Account Management
2,Request: Add bulk operation support to API Gat...,API Gateway,request_router,high,phone,enterprise,NA,New Feature,We would like to request a feature for API Gat...,6757513138608122457,Feature Request


In [ ]:
labels_columns = ['subcategory','category']
X_train = train_df.drop(columns=[labels_columns], errors="ignore")
X_val   = val_df.drop(columns=[labels_columns], errors="ignore")
X_test  = test_df.drop(columns=[labels_columns], errors="ignore")

y_train = train_df['category']
y_val = val_df['category']
y_test = test_df['category']


cat_cols = [
            "product",
            "product_module",
            "priority",
            "channel",
            "customer_tier"
        ]
text_col = "text"

preprocess_pipeline_tfidf = build_feature_encoder_pipeline_tfidf(cat_cols=cat_cols,text_col=text_col)
X_train_transformed = preprocess_pipeline_tfidf.fit_transform(X_train)
X_val_transformed = preprocess_pipeline_tfidf.transform(X_val)
X_test_transformed = preprocess_pipeline_tfidf.transform(X_test)



In [ ]:
X_train.shape

(63498, 11)

In [ ]:
f"training data shape: features: {X_train_transformed.shape}, labels: {y_train.shape}"

'training data shape: features: (63498, 757), labels: (63498,)'

In [ ]:
# tfidf = preprocess.named_transformers_["text"]

# feature_names = tfidf.get_feature_names_out()

# print(len(feature_names))
# print(feature_names)

In [23]:
#preprocess_pipe.get_feature_names_out

### LogisticRegression

In [24]:
logistric_model = LogisticRegression(
        max_iter=2000,
        n_jobs=None,          # keep portable across OS
        class_weight="balanced"  # helps if categories are imbalanced
    )

logistric_model.fit(X=X_train_transformed,y=y_train)
# # Validation
# val_pred = logistric_model.predict(X_val_transformed)
# val_f1 = f1_score(y_val, val_pred, average="weighted")
# print("\n--- VALIDATION ---")
# print("Weighted F1:", round(val_f1, 4))
# print(classification_report(y_val, val_pred, digits=3))

# # Test
# test_pred = logistric_model.predict(X_test_transformed)
# test_f1 = f1_score(y_test, test_pred, average="weighted")
# print("\n--- TEST ---")
# print("Weighted F1:", round(test_f1, 4))
# print(classification_report(y_test, test_pred, digits=3))


,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"fit_intercept fit_intercept: bool, default=TrueSpecifies if a constant (a.k.a. bias or intercept) should beadded to the decision function.",True
,"intercept_scaling intercept_scaling: float, default=1Useful only when the solver `liblinear` is usedand `self.fit_intercept` is set to `True`. In this case, `x` becomes`[x, self.intercept_scaling]`,i.e. a ""synthetic"" feature with constant value equal to`intercept_scaling` is appended to the instance vector.The intercept becomes``intercept_scaling * synthetic_feature_weight``... note:: The synthetic feature weight is subject to L1 or L2 regularization as all other features. To lessen the effect of regularization on synthetic feature weight (and therefore on the intercept) `intercept_scaling` has to be increased.",1
,"class_weight class_weight: dict or 'balanced', default=NoneWeights associated with classes in the form ``{class_label: weight}``.If not given, all classes are supposed to have weight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.Note that these weights will be multiplied with sample_weight (passedthrough the fit method) if sample_weight is specified... versionadded:: 0.17 *class_weight='balanced'*",'balanced'
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag', 'saga' or 'liblinear' to shuffle thedata. See :term:`Glossary ` for details.",None
,"solver solver: {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'}, default='lbfgs'Algorithm to use in the optimization problem. Default is 'lbfgs'.To choose a solver, you might want to consider the following aspects:- 'lbfgs' is a good default solver because it works reasonably well for a wide class of problems.- For :ter

#### XGboost

In [28]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix, hstack

from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder


from xgboost import XGBClassifier

/home/ramesh/Personal_projects/AI-based-Product-Support-System/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [32]:
labels_columns = ['subcategory','category']

train_df = train_df[:500]
val_df = val_df[:100]
test_df = test_df[:100]

X_train = train_df.drop(columns=[labels_columns], errors="ignore")
X_val   = val_df.drop(columns=[labels_columns], errors="ignore")
X_test  = test_df.drop(columns=[labels_columns], errors="ignore")

y_train = train_df['category']
y_val = val_df['category']
y_test = test_df['category']


cat_cols = [
            "product",
            "product_module",
            "priority",
            "channel",
            "customer_tier"
        ]
text_col = "text"

In [34]:
def encode_text(model: SentenceTransformer, texts: list[str], batch_size: int = 64) -> np.ndarray:


    emb = model.encode(
        texts,
        batch_size=batch_size,
        show_progress_bar=True,
        convert_to_numpy=True,
        normalize_embeddings=True,
           device='cpu'   # often improves downstream linear/tree models
    )
    return emb


In [35]:
# --- label encoding ---
le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)
y_val_enc = le.transform(y_val)
y_test_enc = le.transform(y_test)


# --- text embeddings ---
embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

X_train_text = encode_text(embedder, X_train["text"].tolist(), batch_size=256)
X_val_text   = encode_text(embedder, X_val["text"].tolist(), batch_size=256)
X_test_text  = encode_text(embedder, X_test["text"].tolist(), batch_size=256)

Batches: 100%|██████████| 1/1 [00:01<00:00,  1.08s/it]


In [36]:
# Convert dense embeddings to sparse so we can hstack with OneHot output
X_train_text_sp = csr_matrix(X_train_text)
X_val_text_sp   = csr_matrix(X_val_text)
X_test_text_sp  = csr_matrix(X_test_text)

# --- categorical one-hot ---
ohe = OneHotEncoder(handle_unknown="ignore")
X_train_cat = ohe.fit_transform(X_train[cat_cols])
X_val_cat   = ohe.transform(X_val[cat_cols])
X_test_cat  = ohe.transform(X_test[cat_cols])

# --- combine ---
X_train_final = hstack([X_train_text_sp, X_train_cat], format="csr")
X_val_final   = hstack([X_val_text_sp,   X_val_cat],   format="csr")
X_test_final  = hstack([X_test_text_sp,  X_test_cat],  format="csr")

In [31]:
# --- XGBoost ---
xgb = XGBClassifier(
    objective="multi:softprob",
    # num_class=len(le.classes_),
    n_estimators=500,
    max_depth=6,
    learning_rate=0.08,
    subsample=0.9,
    colsample_bytree=0.9,
    reg_lambda=1.0,
    tree_method="hist",
    eval_metric="mlogloss",
    random_state=42,
)




In [ ]:
xgb.fit(X_train_final, y_train_enc)

In [38]:
# VAL
val_pred_enc = xgb.predict(X_val_final)
val_pred = le.inverse_transform(val_pred_enc)
print("VAL weighted F1:", round(f1_score(y_val, val_pred, average="weighted"), 4))
print(classification_report(y_val, val_pred, digits=3))

# TEST
test_pred_enc = xgb.predict(X_test_final)
test_pred = le.inverse_transform(test_pred_enc)
print("TEST weighted F1:", round(f1_score(y_test, test_pred, average="weighted"), 4))
print(classification_report(y_test, test_pred, digits=3))

VAL weighted F1: 0.8772
                    precision    recall  f1-score   support

Account Management      0.000     0.000     0.000         0
        Data Issue      0.909     0.909     0.909        22
   Feature Request      0.941     1.000     0.970        32
          Security      1.000     0.742     0.852        31
   Technical Issue      0.600     0.800     0.686        15

          accuracy                          0.870       100
         macro avg      0.690     0.690     0.683       100
      weighted avg      0.901     0.870     0.877       100

TEST weighted F1: 0.9175
                 precision    recall  f1-score   support

     Data Issue      1.000     0.933     0.966        15
Feature Request      1.000     1.000     1.000        34
       Security      1.000     0.667     0.800        21
Technical Issue      0.789     1.000     0.882        30

       accuracy                          0.920       100
      macro avg      0.947     0.900     0.912       100
   weig

/home/ramesh/Personal_projects/AI-based-Product-Support-System/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1833: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/ramesh/Personal_projects/AI-based-Product-Support-System/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1833: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/ramesh/Personal_projects/AI-based-Product-Support-System/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1833: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` 

In [33]:
from src.classification.feature_builder import BuildClassificationFeatures



In [34]:
# # Category only
# features = BuildClassificationFeatures(
#     json_data[0],
#     label_mode="category"
# ).build(sample_sizes=(500, 100, 100))



# Category + subcategory
features = BuildClassificationFeatures(
    json_data[0],
    label_mode="category|subcategory",
    text_mode='tfidf'
).build(sample_sizes = None)

In [35]:
features.y_test

array([11,  5, 11, ...,  8,  8, 17], shape=(11505,))

In [36]:
features.label_encoder.inverse_transform(features.y_train)

array(['Account Management|Upgrade', 'Feature Request|New Feature',
       'Account Management|Upgrade', ..., 'Account Management|Upgrade',
       'Security|Compliance', 'Data Issue|Sync Error'],
      shape=(63498,), dtype=object)

In [ ]:
# --- XGBoost ---
xgb = XGBClassifier(
    objective="multi:softprob",
    #num_class=len(le.classes_),
    n_estimators=500,
    max_depth=6,
    learning_rate=0.08,
    subsample=0.9,
    colsample_bytree=0.9,
    reg_lambda=1.0,
    tree_method="hist",
    eval_metric="mlogloss",
    random_state=42,
)

xgb.fit(features.X_train, features.y_train)

,"objective objective: typing.Union[str, xgboost.sklearn._SklObjWProto, typing.Callable[[typing.Any, typing.Any], typing.Tuple[numpy.ndarray, numpy.ndarray]], NoneType]Specify the learning task and the corresponding learning objective or a customobjective function to be used.For custom objective, see :doc:`/tutorials/custom_metric_obj` and:ref:`custom-obj-metric` for more information, along with the end note forfunction signatures.",'multi:softprob'
,"base_score base_score: typing.Union[float, typing.List[float], NoneType]The initial prediction score of all instances, global bias.",None
,booster,None
,"callbacks callbacks: typing.Optional[typing.List[xgboost.callback.TrainingCallback]]List of callback functions that are applied at end of each iteration.It is possible to use predefined callbacks by using:ref:`Callback API `... note:: States in callback are not preserved during training, which means callback objects can not be reused for multiple training sessions without reinitialization or deepcopy... code-block:: python for params in parameters_grid: # be sure to (re)initialize the callbacks before each run callbacks = [xgb.callback.LearningRateScheduler(custom_rates)] reg = xgboost.XGBRegressor(**params, callbacks=callbacks) reg.fit(X, y)",None
,colsample_bylevel colsample_bylevel: typing.Optional[float]Subsample ratio of columns for each level.,None
,colsample_bynode colsample_bynode: typing.Optional[float]Subsample ratio of columns for each split.,None
,colsample_bytree colsample_bytree: typing.Optional[float]Subsample ratio of columns when constructing each tree.,0.9
,"device device: typing.Optional[str].. versionadded:: 2.0.0Device ordinal, available options are `cpu`, `cuda`, and `gpu`.",None
,"early_stopping_rounds early_stopping_rounds: typing.Optional[int].. versionadded:: 1.6.0- Activates early stopping. Validation metric needs to improve at least once in every **early_stopping_rounds** round(s) to continue training. Requires at least one item in **eval_set** in :py:meth:`fit`.- If early stopping occurs, the model will have two additional attributes: :py:attr:`best_score` and :py:attr:`best_iteration`. These are used by the :py:meth:`predict` and :py:meth:`apply` methods to determine the optimal number of trees during inference. If users want to access the full model (including trees built after early stopping), they can specify the `iteration_range` in these inference methods. In addition, other utilities like model plotting can also use the entire model.- If you prefer to discard the trees after `best_iteration`, consider using the callback function :py:class:`xgboost.callback.EarlyStopping`.- If there's more than one item in **eval_set**, the last entry will be used for early stopping. If there's more than one metric in **eval_metric**, the last metric will be used for early stopping.",None
,enable_categorical enable_categorical: boolSee the same parameter of :py:class:`DMatrix` for details.,False
,"eval_metric eval_metric: typing.Union[str, typing.List[typing.Union[str, typing.Callable]], typing.Callable, NoneType].. versionadded:: 1.6.0Metric used for monitoring the training result and early stopping. It can be astring or list of strings as names of predefined metric in XGBoost (See:doc:`/parameter`), one of the metrics in :py:mod:`sklearn.metrics`, or anyother user defined metric that looks like `sklearn.metrics`.If custom objective is also provided, then custom metric should implement thecorresponding reverse link function.Unlike the `scoring` parameter commonly used in scikit-learn, when a callableobject is provided, it's assumed to be a cost function and by default XGBoostwill minimize the result during early stopping.For advanced usage on Early stopping like directly choosing to maximize insteadof minimize, see :py:obj:`xgboost.callback.EarlyStopping`.See :doc:`/tutorials/custom_metric_obj` and :ref:`custom-obj-metric` for moreinformation... code-block:: python from sklearn.datasets import load_diabetes fr

In [38]:
# VAL
val_pred_enc = xgb.predict(features.X_val)
val_pred = features.label_encoder.inverse_transform(val_pred_enc)
val_original = features.label_encoder.inverse_transform(features.y_val)

print("VAL weighted F1:", round(f1_score(val_original, val_pred, average="weighted"), 4))
print(classification_report(val_original, val_pred, digits=3))

# # TEST
# test_pred_enc = xgb.predict(features.X_test)
# test_pred = features.label_encoder.inverse_transform(test_pred_enc)
# print("TEST weighted F1:", round(f1_score(features.y_test, test_pred, average="weighted"), 4))
# print(classification_report(features.y_test, test_pred, digits=3))

VAL weighted F1: 0.198
                               precision    recall  f1-score   support

        Data Issue|Corruption      0.196     0.223     0.209       533
         Data Issue|Data Loss      0.209     0.207     0.208       565
     Data Issue|Import/Export      0.223     0.220     0.221       555
        Data Issue|Sync Error      0.196     0.185     0.190       573
        Data Issue|Validation      0.205     0.195     0.200       585
          Feature Request|API      0.205     0.161     0.180       665
Feature Request|Documentation      0.208     0.146     0.171       692
  Feature Request|Enhancement      0.174     0.174     0.174       639
  Feature Request|New Feature      0.186     0.215     0.200       661
        Feature Request|UI/UX      0.198     0.269     0.228       691
      Security|Authentication      0.200     0.319     0.246       520
       Security|Authorization      0.185     0.185     0.185       497
          Security|Compliance      0.197     0.163   

In [ ]:
def train_mode_for_category_prediction(json_data):
    features = BuildClassificationFeatures(
    json_data[0],
    label_mode="category|subcategory",
    text_mode='tfidf'
        ).build(sample_sizes = None)
    
        # --- XGBoost ---
    xgb = XGBClassifier(
        objective="multi:softprob",
        #num_class=len(le.classes_),
        n_estimators=500,
        max_depth=6,
        learning_rate=0.08,
        subsample=0.9,
        colsample_bytree=0.9,
        reg_lambda=1.0,
        tree_method="hist",
        eval_metric="mlogloss",
        random_state=42,
    )

    xgb.fit(features.X_train, features.y_train)

        # VAL
    val_pred_enc = xgb.predict(features.X_val)
    val_pred = features.label_encoder.inverse_transform(val_pred_enc)
    val_original = features.label_encoder.inverse_transform(features.y_val)

    print("VAL weighted F1:", round(f1_score(val_original, val_pred, average="weighted"), 4))
    print(classification_report(val_original, val_pred, digits=3))

    # TEST
    test_pred_enc = xgb.predict(features.X_test)
    test_pred = features.label_encoder.inverse_transform(test_pred_enc)
    tesr_original = features.label_encoder.inverse_transform(features.y_test)

    print("TEST weighted F1:", round(f1_score(tesr_original, test_pred, average="weighted"), 4))
    print(classification_report(tesr_original, test_pred, digits=3))



In [39]:
filtered_tickets = [
    ticket for ticket in json_data[0]
    if ticket.category == "Feature Request"
]


In [63]:
len(filtered_tickets)
# Category only
# Category + subcategory
features = BuildClassificationFeatures(
    filtered_tickets,
    label_mode="subcategory",
    text_mode='transformer'
).build(sample_sizes = None)

Batches: 100%|██████████| 14/14 [00:37<00:00,  2.65s/it]


In [64]:
features.y_train

array([0, 2, 0, ..., 2, 0, 3], shape=(10150,))

In [65]:
features.X_train

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 3948350 stored elements and shape (10150, 414)>

In [66]:
xgb.fit(features.X_train, features.y_train)

,"objective objective: typing.Union[str, xgboost.sklearn._SklObjWProto, typing.Callable[[typing.Any, typing.Any], typing.Tuple[numpy.ndarray, numpy.ndarray]], NoneType]Specify the learning task and the corresponding learning objective or a customobjective function to be used.For custom objective, see :doc:`/tutorials/custom_metric_obj` and:ref:`custom-obj-metric` for more information, along with the end note forfunction signatures.",'multi:softprob'
,"base_score base_score: typing.Union[float, typing.List[float], NoneType]The initial prediction score of all instances, global bias.",None
,booster,None
,"callbacks callbacks: typing.Optional[typing.List[xgboost.callback.TrainingCallback]]List of callback functions that are applied at end of each iteration.It is possible to use predefined callbacks by using:ref:`Callback API `... note:: States in callback are not preserved during training, which means callback objects can not be reused for multiple training sessions without reinitialization or deepcopy... code-block:: python for params in parameters_grid: # be sure to (re)initialize the callbacks before each run callbacks = [xgb.callback.LearningRateScheduler(custom_rates)] reg = xgboost.XGBRegressor(**params, callbacks=callbacks) reg.fit(X, y)",None
,colsample_bylevel colsample_bylevel: typing.Optional[float]Subsample ratio of columns for each level.,None
,colsample_bynode colsample_bynode: typing.Optional[float]Subsample ratio of columns for each split.,None
,colsample_bytree colsample_bytree: typing.Optional[float]Subsample ratio of columns when constructing each tree.,0.9
,"device device: typing.Optional[str].. versionadded:: 2.0.0Device ordinal, available options are `cpu`, `cuda`, and `gpu`.",None
,"early_stopping_rounds early_stopping_rounds: typing.Optional[int].. versionadded:: 1.6.0- Activates early stopping. Validation metric needs to improve at least once in every **early_stopping_rounds** round(s) to continue training. Requires at least one item in **eval_set** in :py:meth:`fit`.- If early stopping occurs, the model will have two additional attributes: :py:attr:`best_score` and :py:attr:`best_iteration`. These are used by the :py:meth:`predict` and :py:meth:`apply` methods to determine the optimal number of trees during inference. If users want to access the full model (including trees built after early stopping), they can specify the `iteration_range` in these inference methods. In addition, other utilities like model plotting can also use the entire model.- If you prefer to discard the trees after `best_iteration`, consider using the callback function :py:class:`xgboost.callback.EarlyStopping`.- If there's more than one item in **eval_set**, the last entry will be used for early stopping. If there's more than one metric in **eval_metric**, the last metric will be used for early stopping.",None
,enable_categorical enable_categorical: boolSee the same parameter of :py:class:`DMatrix` for details.,False
,"eval_metric eval_metric: typing.Union[str, typing.List[typing.Union[str, typing.Callable]], typing.Callable, NoneType].. versionadded:: 1.6.0Metric used for monitoring the training result and early stopping. It can be astring or list of strings as names of predefined metric in XGBoost (See:doc:`/parameter`), one of the metrics in :py:mod:`sklearn.metrics`, or anyother user defined metric that looks like `sklearn.metrics`.If custom objective is also provided, then custom metric should implement thecorresponding reverse link function.Unlike the `scoring` parameter commonly used in scikit-learn, when a callableobject is provided, it's assumed to be a cost function and by default XGBoostwill minimize the result during early stopping.For advanced usage on Early stopping like directly choosing to maximize insteadof minimize, see :py:obj:`xgboost.callback.EarlyStopping`.See :doc:`/tutorials/custom_metric_obj` and :ref:`custom-obj-metric` for moreinformation... code-block:: python from sklearn.datasets import load_diabetes fr

In [67]:
# VAL
val_pred_enc = xgb.predict(features.X_val)
val_pred = features.label_encoder.inverse_transform(val_pred_enc)
val_original = features.label_encoder.inverse_transform(features.y_val)

print("VAL weighted F1:", round(f1_score(val_original, val_pred, average="weighted"), 4))
print(classification_report(val_original, val_pred, digits=3))

# TEST
test_pred_enc = xgb.predict(features.X_test)
test_pred = features.label_encoder.inverse_transform(test_pred_enc)
tesr_original = features.label_encoder.inverse_transform(features.y_test)

print("TEST weighted F1:", round(f1_score(tesr_original, test_pred, average="weighted"), 4))
print(classification_report(tesr_original, test_pred, digits=3))

VAL weighted F1: 0.1888
               precision    recall  f1-score   support

          API      0.192     0.207     0.199       691
Documentation      0.151     0.077     0.102       650
  Enhancement      0.209     0.161     0.182       660
  New Feature      0.180     0.182     0.181       687
        UI/UX      0.222     0.352     0.272       708

     accuracy                          0.198      3396
    macro avg      0.191     0.196     0.187      3396
 weighted avg      0.191     0.198     0.189      3396

TEST weighted F1: 0.1887
               precision    recall  f1-score   support

          API      0.163     0.066     0.094       708
Documentation      0.213     0.177     0.193       661
  Enhancement      0.199     0.296     0.238       669
  New Feature      0.200     0.227     0.213       661
        UI/UX      0.195     0.231     0.212       650

     accuracy                          0.198      3349
    macro avg      0.194     0.199     0.190      3349
 weighted a

In [59]:
val_pred

array(['Enhancement', 'New Feature', 'Documentation', ...,
       'Documentation', 'API', 'API'], shape=(3396,), dtype=object)

In [60]:
val_original

array(['New Feature', 'API', 'Documentation', ..., 'Enhancement', 'API',
       'Documentation'], shape=(3396,), dtype=object)

In [47]:
df_data.category.unique()

array(['Feature Request', 'Account Management', 'Security', 'Data Issue',
       'Technical Issue'], dtype=object)

In [62]:
def create_model_for_each_category():
    # categories = ['Feature Request', 'Account Management', 'Security', 'Data Issue',
    #    'Technical Issue']

    categories = ['Feature Request', 'Account Management', 'Security']
    
    for category in categories:
        filtered_tickets = [
         ticket for ticket in json_data[0]
         if ticket.category == category]
        
        print("number of ticktes in category {}: {}".format(category,len(filtered_tickets)))

        features = BuildClassificationFeatures(
            filtered_tickets,
            label_mode="subcategory",
            text_mode='tfidf'
        ).build(sample_sizes = None)

        xgb.fit(features.X_train, features.y_train)

        # VAL
        val_pred_enc = xgb.predict(features.X_val)
        val_pred = features.label_encoder.inverse_transform(val_pred_enc)
        val_original = features.label_encoder.inverse_transform(features.y_val)

        print("VAL weighted F1:", round(f1_score(val_original, val_pred, average="weighted"), 4))
        print(classification_report(val_original, val_pred, digits=3))

        # TEST
        test_pred_enc = xgb.predict(features.X_test)
        test_pred = features.label_encoder.inverse_transform(test_pred_enc)
        tesr_original = features.label_encoder.inverse_transform(features.y_test)

        print("TEST weighted F1:", round(f1_score(tesr_original, test_pred, average="weighted"), 4))
        print(classification_report(tesr_original, test_pred, digits=3))

create_model_for_each_category()

number of ticktes in category Feature Request: 22047
VAL weighted F1: 0.1986
               precision    recall  f1-score   support

          API      0.199     0.200     0.199       691
Documentation      0.189     0.226     0.206       650
  Enhancement      0.203     0.174     0.187       660
  New Feature      0.197     0.108     0.139       687
        UI/UX      0.223     0.309     0.259       708

     accuracy                          0.204      3396
    macro avg      0.202     0.203     0.198      3396
 weighted avg      0.202     0.204     0.199      3396

TEST weighted F1: 0.1935
               precision    recall  f1-score   support

          API      0.199     0.196     0.197       708
Documentation      0.203     0.244     0.222       661
  Enhancement      0.197     0.161     0.178       669
  New Feature      0.194     0.104     0.136       661
        UI/UX      0.198     0.291     0.236       650

     accuracy                          0.199      3349
    macro avg